In [0]:
import numpy as np

In [0]:
# -P For download ing/ saving file at your custom directory and not in the same directory where this code is running
!pip install wget

In [0]:
! wget http://www.manythings.org/anki/deu-eng.zip 

--2019-11-17 08:33:16--  http://www.manythings.org/anki/deu-eng.zip
Resolving www.manythings.org (www.manythings.org)... 104.24.109.196, 104.24.108.196, 2606:4700:30::6818:6dc4, ...
Connecting to www.manythings.org (www.manythings.org)|104.24.109.196|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7612057 (7.3M) [application/zip]
Saving to: ‘deu-eng.zip.1’

deu-eng.zip.1       100%[===================>]   7.26M  3.82MB/s    in 1.9s    

2019-11-17 08:33:19 (3.82 MB/s) - ‘deu-eng.zip.1’ saved [7612057/7612057]



In [0]:
!unzip deu-eng.zip

Archive:  deu-eng.zip
replace deu.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: deu.txt                 
replace _about.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: _about.txt              


In [0]:
file = open("deu.txt", mode="rt", encoding="utf-8")
data = file.read()

In [0]:
def to_lines(text):
  sent = text.strip().split("\n") # Every lines is split up 
  sent = [r.split("\t")  for r in sent ]
  return sent         

In [0]:
deu_eng = to_lines(data)
deu_eng = np.array(deu_eng) 
deu_eng[0]                       

array(['Hi.', 'Hallo!',
       'CC-BY 2.0 (France) Attribution: tatoeba.org #538123 (CM) & #380701 (cburgmer)'],
      dtype='<U537')

In [0]:
deu_eng =  deu_eng[:50000,:]

In [0]:
import string 
deu_eng[:,0] = [s.translate(str.maketrans('','',string.punctuation)).lower() for s in deu_eng[:,0]] 
deu_eng[:,1] = [s.translate(str.maketrans('','',string.punctuation)).lower() for s in deu_eng[:,1]] 


In [0]:
deu_eng

array([['hi', 'hallo',
        'CC-BY 2.0 (France) Attribution: tatoeba.org #538123 (CM) & #380701 (cburgmer)'],
       ['hi', 'grüß gott',
        'CC-BY 2.0 (France) Attribution: tatoeba.org #538123 (CM) & #659813 (Esperantostern)'],
       ['run', 'lauf',
        'CC-BY 2.0 (France) Attribution: tatoeba.org #906328 (papabear) & #941078 (Fingerhut)'],
       ...,
       ['i wholeheartedly agree', 'ich stimme rückhaltlos zu',
        'CC-BY 2.0 (France) Attribution: tatoeba.org #1488273 (Spamster) & #1693172 (al_ex_an_der)'],
       ['i will always love you', 'ich werde dich immer lieben',
        'CC-BY 2.0 (France) Attribution: tatoeba.org #853146 (piksea) & #395302 (xtofu80)'],
       ['i will be back by nine', 'um neun bin ich wieder zurück',
        'CC-BY 2.0 (France) Attribution: tatoeba.org #72281 (CK) & #345033 (lilygilder)']],
      dtype='<U537')

In [0]:
# Tokenizing all the word (split words from sentences to get induvidual words)
from keras.preprocessing.text import Tokenizer

  # check this code from the link sir will provide



In [0]:
def tokenization(line):
  tokenizer = Tokenizer()
  tokenizer.fit_on_texts(line)
  return tokenizer


In [0]:

eng_tokenizer =  tokenization(deu_eng[:,0])
deu_tokenizer = tokenization(deu_eng[:,1])

deu_vocab_size = len(deu_tokenizer.word_index) + 1
eng_vocab_size = len(eng_tokenizer.word_index) + 1

print(len(eng_tokenizer.word_index) + 1)
print(len(deu_tokenizer.word_index)+1)

6361
10597


In [0]:
from keras_preprocessing.sequence import pad_sequences

def encode_sequences(tokenizer, length, lines):
  seq = tokenizer.texts_to_sequences(lines)
  seq = pad_sequences(seq, maxlen = length, padding= "post")
  return seq

In [0]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(deu_eng, test_size = 0.2, random_state = 0)




In [0]:
trainX = encode_sequences(deu_tokenizer,8,train[:, 1])
trainY = encode_sequences(eng_tokenizer,8,train[:, 0])

testX = encode_sequences(deu_tokenizer,8,test[:, 1])
testY = encode_sequences(eng_tokenizer,8,test[:, 0])

print(trainX.shape)
print(trainX[0])
print(trainY.shape)
print(trainY[0])

(40000, 8)
[ 182   30 3286  783    0    0    0    0]
(40000, 8)
[  92 1255 2610    0    0    0    0    0]


In [0]:
from keras.models import Sequential
from keras.layers import Dense,LSTM,Embedding,RepeatVector


In [0]:
from keras import optimizers

model = Sequential()
model.add(Embedding(deu_vocab_size, input_length=8 ,output_dim=512))
model.add(LSTM(512)) # Thought Vector
model.add(RepeatVector(8)) # Repeat ho raha hai 8 baari 
model.add(LSTM(512, return_sequences=True )) # GRU layer Ki Jagah LSTM assume Kar lo abhi ke liye 
model.add(Dense( eng_vocab_size,activation = 'softmax'))
model.compile(optimizer= optimizers.RMSprop(lr=0.001), loss='sparse_categorical_crossentropy')


In [0]:
from keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint("model.h1 ",
                             monitor="val_loss",
                             verbose=1, 
                             save_best_only=True,
                             mode="min")

history = model.fit(trainX,
                    trainY.reshape(trainY.shape[0],trainY.shape[1], 1),
                    epochs=30,
                    batch_size = 512,
                    validation_split =0.2,
                    callbacks = [checkpoint],
                    verbose = 1)



Train on 32000 samples, validate on 8000 samples
Epoch 1/30
32000/32000 [==============================] - 17s 542us/step - loss: 3.3877 - val_loss: 2.9232

Epoch 00001: val_loss improved from inf to 2.92320, saving model to model.h1 
Epoch 2/30
32000/32000 [==============================] - 15s 463us/step - loss: 2.8793 - val_loss: 2.8044

Epoch 00002: val_loss improved from 2.92320 to 2.80439, saving model to model.h1 
Epoch 3/30
32000/32000 [==============================] - 15s 464us/step - loss: 2.7110 - val_loss: 2.7132

Epoch 00003: val_loss improved from 2.80439 to 2.71316, saving model to model.h1 
Epoch 4/30
32000/32000 [==============================] - 15s 463us/step - loss: 2.5802 - val_loss: 2.6011

Epoch 00004: val_loss improved from 2.71316 to 2.60110, saving model to model.h1 
Epoch 5/30
32000/32000 [==============================] - 15s 463us/step - loss: 2.4494 - val_loss: 2.5030

Epoch 00005: val_loss improved from 2.60110 to 2.50302, saving model to model.h1 
Epoch

In [0]:
from keras.models import load_model

models = load_model("model.h1")
preds = models.predict_classes(testX)